<a href="https://colab.research.google.com/github/DavidSchineis/Math-Physics/blob/main/Copy_of_Lab_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Abstract
This lab explores matrix transformations and basic image compression using Python. We began by applying transformations such as rotation, scaling, and sheering to varying shapes and images, visualizing how these transformations happen and how the order of operations effects the result. Then, we used np.diag to diagonalize matrices, verifying the relationship between a matrix, eigenvalues, and eigenvectors. Then, we brought these concepts to image compression by projecting an image onto a set of eigenvectors from the k largest eigenvalues. This lab demonstrates how linear algebra principles are at the core of image transformations, diagonalization, and compression in Python.

In [ ]:
import numpy as np
from numpy import pi, cos, sin, tan, cosh, sinh, tanh
from PIL import Image
import requests
import matplotlib.pyplot as plt

url='https://upload.wikimedia.org/wikipedia/commons/thumb/8/83/Solar_system.jpg/193px-Solar_system.jpg'
headers ={
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
}
x=requests.get(url, stream=True,headers=headers)

# Load the image
image = Image.open(x.raw)

# Convert the image to a NumPy array
image_array = np.array(image)

display(image)

We can transform this image in a variety of different ways. Rotation can be achieved through multiplying the transformation matrix of
$$ \begin{bmatrix}
\cos(\theta) & -\sin(\theta)\\
\sin(\theta) & \cos(\theta)
\end{bmatrix} $$
and your initial set of points. Before we apply it to an image, we will start with a simple example.

In [ ]:
arr=np.array([[0,3,4,7,0],[0,4,2,5,0]])
theta=np.radians(30)

#express rotation matrix that depends on theta, keep its format as numpy array
transformation_matrix = np.array([[cos(theta), -sin(theta)],[sin(theta),cos(theta)]])

#apply the rotation matrix to arr
new_arr= transformation_matrix @ arr

plt.plot(arr[0],arr[1],label='original')
plt.plot(new_arr[0],new_arr[1],label='transformed')
plt.legend()

plt.gca().set_aspect('equal', adjustable='box')

### Caption
Original shape formed by array and its transformed version which is rotated by 30 degrees.

Other operations are scaling, and shear, which can be achieved through
$$ \begin{bmatrix}
k_1 & 0\\
0 & k_2
\end{bmatrix} $$
for scaling, where $k_1$ is the scaling in $\hat{x}$, and $k_2$ is the scaling in $\hat{y}$. And, for shear
$$ \begin{bmatrix}
1 & m_1\\
m_2 & 1
\end{bmatrix} $$
where $m_1$ is horizontal shear, and $m_2$ is vertical shear.

Repeat the above, applying different transformations, plotting them alongside one another.

In [ ]:
arr=np.array([[0,3,4,7,0],[0,4,2,5,0]])
k1 = 2
k2 = 3
transformation_matrix = np.array([[k1,0],[0,k2]])
scale_arr=transformation_matrix @ arr





arr=np.array([[0,3,4,7,0],[0,4,2,5,0]])
m1 = 2
m2 = 3
transformation_matrix = np.array([[1,m1],[m2,1]])
sheer_arr=transformation_matrix @ arr


plt.plot(arr[0],arr[1],label='original')
plt.plot(scale_arr[0],scale_arr[1],label='scaled')
plt.plot(sheer_arr[0],sheer_arr[1],label='sheered')
plt.legend()
plt.gca().set_aspect('equal', adjustable='box')

### Caption
Original shape formed by array and two different transformations of it. One where it is being scaled and one where it is being sheered.

Similarly to the simple geometric shapes, we can also transform more complex images.

Although this cell has quite a bit of code, the only thing you need to make it working is to express the approriate transformation matrix that performs rotation.

In [ ]:
# Define the angle of rotation in degrees
angle = 30
theta= np.radians(angle)

#express rotation matrix that depends on theta, keep its format as numpy array
transformation_matrix = np.array([[cos(theta), -sin(theta)],[sin(theta),cos(theta)]])


# Calculating the dimensions of the transformed image
height, width = image_array.shape[:2]
corners = np.array([[0, 0], [width, 0], [width, height], [0, height]])
transformed_corners = np.dot(corners, transformation_matrix.T)
new_width = int(np.ceil(np.max(transformed_corners[:, 0])) - np.floor(np.min(transformed_corners[:, 0])))
new_height = int(np.ceil(np.max(transformed_corners[:, 1])) - np.floor(np.min(transformed_corners[:, 1])))

# Create an empty array to store the transformed image
transformed_image_array = np.zeros((new_height, new_width, 3), dtype=np.uint8)+255

# Calculate the center of the original and transformed images
original_center = np.array([width / 2, height / 2])
transformed_center = np.array([new_width / 2, new_height / 2])

# Perform the transformation
for i in range(height):
    for j in range(width):
        # Calculate the position relative to the original center
        position = np.array([j, i]) - original_center
        # Apply the transformation using matrix multiplication
        transformed_position = np.dot(np.linalg.inv(transformation_matrix), position)
        # Calculate the position relative to the transformed center
        transformed_position += transformed_center
        # Convert the position to integer values
        transformed_position = np.round(transformed_position).astype(int)
        # Check if the transformed position is within the bounds of the transformed image
        if 0 <= transformed_position[1] < new_height and 0 <= transformed_position[0] < new_width:
            transformed_image_array[transformed_position[1], transformed_position[0]] = image_array[i,j]

# Convert the transformed image array back to PIL image
transformed_image = Image.fromarray(transformed_image_array)

# Display the original and transformed images
display(image)
display(transformed_image)

#### Question

Describe what the code above is doing step by step, in your own words.

#### Answer
We build a rotation matrix as an array which has been given an angle of rotation of 30 degrees. We then use the images shape to find its width and height. From its width and height, we find its corners where we apply the rotation matrix. From these new corners, we can find a new width and height. From the new width and height, we can compare a new center with the old. Then, we iterate through the new width and heights to check each position and apply the rotation matrix then convert the position value to an integer and convert it back to an image.

----
In the image above, every single pixel has been faithfully moved to a new location, but a finite number of pixels moved onto a larger grid, the result is sub-optimal. As such, it would be better to find not the mapping of the original image onto a new grid, but rather to map every single pixel of the new image onto the coordinates of the original image.

Copy the above code and modify the for loop to ensure there are no dead pixels in the rotated image. I.e., instead of iterating across every single pixel of the *original* image, we should instead iterate across every pixel of the *transformed* image.


Remember that the rotation is defined in counterclockwise direction, and your new "fixed" rotated image should have the same orientation as the rotated image above.

In [ ]:

# Define the angle of rotation in degrees
angle = 30
theta= np.radians(360-angle)

#express rotation matrix that depends on theta, keep its format as numpy array
transformation_matrix = np.array([[cos(theta), -sin(theta)],[sin(theta),cos(theta)]])


# Calculating the dimensions of the transformed image
height, width = image_array.shape[:2]
corners = np.array([[0, 0], [width, 0], [width, height], [0, height]])
transformed_corners = np.dot(corners, transformation_matrix.T)
new_width = int(np.ceil(np.max(transformed_corners[:, 0])) - np.floor(np.min(transformed_corners[:, 0])))
new_height = int(np.ceil(np.max(transformed_corners[:, 1])) - np.floor(np.min(transformed_corners[:, 1])))

# Create an empty array to store the transformed image
transformed_image_array = np.zeros((new_height, new_width, 3), dtype=np.uint8)+255

# Calculate the center of the original and transformed images
original_center = np.array([width / 2, height / 2])
transformed_center = np.array([new_width / 2, new_height / 2])

# Perform the transformation
for i in range(new_height):
    for j in range(new_width):
        # Calculate the position relative to the original center
        position = np.array([j, i]) - transformed_center
        # Apply the inverse transformation using matrix multiplication
        original_position = np.dot(np.linalg.inv(transformation_matrix), position)
        # Calculate the position relative to the transformed center
        original_position += original_center
        # Convert the position to integer values
        original_position = np.round(original_position).astype(int)
        # Check if the transformed position is within the bounds of the transformed image
        if 0 <= original_position[1] < height and 0 <= original_position[0] < width:
            transformed_image_array[i, j] = image_array[original_position[1], original_position[0]]

# Convert the transformed image array back to PIL image
transformed_image = Image.fromarray(transformed_image_array)

# Display the original and transformed images
display(image)
display(transformed_image)

Instead of rotation, scale the image up by a factor of 1.5, and apply horizontal sheer of 0.3 (shear should skew image to the right by convention, as you can check doing this in the first part of this excersise)

In [ ]:
k = 1.5
m = 0.3

scale = np.array([[k, 0],[0, k]])
sheer = np.array([[1, m],[0, 1]])

transformation_matrix = scale @ sheer

# Calculating the dimensions of the transformed image
height, width = image_array.shape[:2]
corners = np.array([[0, 0], [width, 0], [width, height], [0, height]])
transformed_corners = np.dot(corners, transformation_matrix.T)
new_width = int(np.ceil(np.max(transformed_corners[:, 0])) - np.floor(np.min(transformed_corners[:, 0])))
new_height = int(np.ceil(np.max(transformed_corners[:, 1])) - np.floor(np.min(transformed_corners[:, 1])))

# Create an empty array to store the transformed image
transformed_image_array = np.zeros((new_height, new_width, 3), dtype=np.uint8)+255

# Calculate the center of the original and transformed images
original_center = np.array([width / 2, height / 2])
transformed_center = np.array([new_width / 2, new_height / 2])

# Perform the transformation
for i in range(new_height):
    for j in range(new_width):
        # Calculate the position relative to the original center
        position = np.array([j, i]) - transformed_center
        # Apply the inverse transformation using matrix multiplication
        original_position = np.dot(np.linalg.inv(transformation_matrix), position)
        # Calculate the position relative to the transformed center
        original_position += original_center
        # Convert the position to integer values
        original_position = np.round(original_position).astype(int)
        # Check if the transformed position is within the bounds of the transformed image
        if 0 <= original_position[1] < height and 0 <= original_position[0] < width:
            transformed_image_array[i, j] = image_array[original_position[1], original_position[0]]

# Convert the transformed image array back to PIL image
transformed_image = Image.fromarray(transformed_image_array)

# Display the original and transformed images
display(image)
display(transformed_image)




Now combine all 3 transformations in a singlt transformation matrix: rotation of 90 degrees, scale the image up by a factor of 1.5, and apply horizontal sheer of 0.3. How does the image change depending on the order of operations of rotation and shear?
#### Answer
Scale can go anywhere without seeming to effect the shape of the image but sheer and rotation effect each other with their order. If you sheer first, the sheer rotates with the image and sheers vertically. If you rotate first, the rotation happens first so you get a vertically sheered image but its rotated in the horizontal.

# Diagonalization

In working with matrices, it is often very useful to diagonalize a matrix. Diagonalization is a powerful technique that allows us to simplify calculations and gain insights into the behavior of a matrix. It involves transforming a square matrix into a diagonal matrix using eigenvectors and eigenvalues.

For example, let $$A=\begin{bmatrix}
a_{11} & a_{12}\\
a_{21} & a_{22}
\end{bmatrix}$$

For this matrix $A$ there exists such a matrix $D$ that corresponds to the eigenvalues of $A$ on the diagonal
$$D=\begin{bmatrix}
\lambda_1 & 0\\
0 & \lambda_2
\end{bmatrix}$$
as well as matrix $P$ consisting of the eigenvectors corresponding to these eigenvalues
$$P=\begin{bmatrix} \vec{v_1} & \vec{v_2} \end{bmatrix}=\begin{bmatrix}
v_{11} & v_{21}\\
v_{12} & v_{22}
\end{bmatrix}$$

Then, if $P$ is invertible, $A=PDP^{-1}$, but also, $D=P^{-1}AP$. We can test it here.

In [ ]:
A=np.matrix([[7,2],[-4,1]])
eigenvalues=np.array([5,3])
eigenvectors= np.matrix([[1,1],[-1,-2]])

D=np.diag(eigenvalues)

print(A-np.round(eigenvectors*D*np.linalg.inv(eigenvectors),5))
print(D-np.round(np.linalg.inv(eigenvectors)*A*eigenvectors,5))

Rather than doing it by hand, you can use np.linalg.eig to find eigenvalues and eigenvectors of a matrix. Use them to check your work - remember that eigenvectors can be scaled, and thus are not unique.

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(A)
D=np.diag(eigenvalues)

print(A-np.round(eigenvectors*D*np.linalg.inv(eigenvectors),5))
print(D-np.round(np.linalg.inv(eigenvectors)*A*eigenvectors,5))

Test diagonalization and reconstruction of a matrix using a much larger matrix, e.g., 100x100. You can use np.random.random() to generate values of the matrix

In [ ]:
A = np.matrix(np.random.random((100,100)))
eigenvalues, eigenvectors = np.linalg.eig(A)
D=np.diag(eigenvalues)

plt.hist(np.array(A-eigenvectors*D*np.linalg.inv(eigenvectors)).flatten())
plt.show()

Thus, you can store either the original matrix, or a combination of its eigenvalues and eigenvectors (and reconstruct the original afterwards), but since the set of eigenvectors has the same shape as the original matrix, this would not save any space on your hard drive.

It is possible, however, create a very simple compression algorithm using a very similar approach. This code can be ran as is.

In [ ]:
# Convert image to grayscale
image_gray = np.mean(image, axis=2)

# Compute the covariance matrix
covariance_matrix = np.cov(image_gray.T)

# Compute the eigenvectors and eigenvalues
eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

# Sort the eigenvectors based on eigenvalues
sorted_indices = np.flip(np.argsort(eigenvalues))
eigenvalues = eigenvalues[sorted_indices]
eigenvectors = eigenvectors[:, sorted_indices]

# Set the number of eigenvectors (k) to keep
k = 30

# Select the top k eigenvectors
selected_eigenvectors = eigenvectors[:, :k]

# Project the image onto the selected eigenvectors
compressed_image = np.dot(image_gray, selected_eigenvectors)

# Reconstruct the image using the selected eigenvectors
reconstructed_image = np.dot(compressed_image, selected_eigenvectors.T)

# Plot original and reconstructed images
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs[0].imshow(image_gray, cmap='gray')
axs[0].set_title('Original Image')
axs[0].axis('off')
axs[1].imshow(reconstructed_image, cmap='gray')
axs[1].set_title(f'Reconstructed Image (k={k})')
axs[1].axis('off')
axs[2].imshow(image_gray-reconstructed_image, cmap='gray')
axs[2].set_title(f'Difference (k={k})')
axs[2].axis('off')

# Display the plot
plt.tight_layout()
plt.show()


#### Question

Describe what the code above is doing step by step, in your own words.

#### Answer
The code first converts tbe image to grayscale then makes a covariance matrix of the grayscale pixel columns and computes the eigenvalues and eigenvectors of it. Then we sort the eigenvectors by eigenvalue and only keep a specified number. We then dot the grayscale image with the kept eigenvectors. Then we dot that image with the kept eigenvectors transposed. Then we plot the original, the reconstructed image, and their difference to visualize the difference with the amount of selected eigenvectors to keep.

----
#### Question
Why was it important to sort eigenvalues prior to discarding all but 30 of them?
Experiment what would happen if you were to set
sorted_indices = np.random.permutation(np.arange(len(eigenvalues)))
That is to say, rather than selecting 30 largest eigenvalues, you would pick any 30 eigenvalues in a random order.

#### Answer

Sorting the eigenvalues to select the 30 largest ones instead of 30 random ones is important as the most varying parts of the image are going to correspond to the largest eigenvalues. The most varying parts of the image are more important to its reconstruction than random features.

----
Use .size or .shape property of numpy arrays to see the number of pixels that are present in the original image. Consider what variables we would need to save in order to produce this image again in compressed form. Assuming the same data format between all the variables, how much smaller would the compressed file be with k=30? What would you consider to be the smallest k to compress the image without significant loses to visual quality, and what would be the compression in this case?

#### Answer
To reproduce the image again in compressed form, we would need to have the k eigenvectors and the image greyscaled. At k=30, the compressed file is 3.6x smaller than the original or 72% reduced. In my opinion, somewhere between k=5 and k=10 makes this specific image no longer comprehendable as the original image.

In [ ]:
H,W = image_gray.shape
original = image_gray.size
compressed = k * (H + W)


print("H, W:", H, W)
print("Original:", original)
print("Compressed:", compressed)
print("Original / Compressed:", original/compressed)


## Extra credit

In 3d, rotation matrices can be expressed as

$$R_x(\theta)= \begin{bmatrix}
1 & 0 & 0 \\
0 & \cos\theta & -\sin(\theta\\
0 & \sin\theta & \cos(\theta
\end{bmatrix},
R_y(\theta)= \begin{bmatrix}
\cos\theta & 0 & \sin\theta\\
0 & 1 & 0 \\
-\sin\theta &0 & \cos\theta
\end{bmatrix},
R_z(\theta)= \begin{bmatrix}
\cos\theta & -\sin\theta & 0\\
\sin\theta & \cos\theta & 0 \\
0 & 0 & 1
\end{bmatrix}$$

You may remember a previous activity where we have created a 3d plot of a sea shell.

Modify the code below to rotate the sea shell around x axis by 60 degrees.

In [ ]:
import plotly.graph_objects as go
u = np.linspace(0, 2*np.pi, 51)
v = np.linspace(-2*np.pi, 2*np.pi, 101)
uGrid, vGrid = np.meshgrid(u,v)


x_orig=5/4.*(1-vGrid/(2*np.pi))*np.cos(2*vGrid)*(1+np.cos(uGrid))+np.cos(2*vGrid)
y_orig=5/4.*(1-vGrid/(2*np.pi))*np.sin(2*vGrid)*(1+np.cos(uGrid))+np.sin(2*vGrid)
z_orig=10*vGrid/(2*np.pi)+5/4.*(1-vGrid/(2*np.pi))*np.sin(uGrid)+15

fig = go.Figure()
fig.add_trace(go.Surface(x=x_orig,
                         y=y_orig,
                         z=z_orig))
fig.show()